# Code investigation

This was done to:
- Split the original query into control and exposed households extraction and see if these bring the same numbers as the original one. 
- Explore how to bring date variables from source - this is still not yet completed
- Add the classification of 'visited' for both control and exposed groups, queries added in request: 
    - Control group 
    <font size="1">

     ```
        hh_control_visited as (
        select
            ctrl.mapped_tv2_hhid 
            ,(case when visited.mapped_tv2_hhid is not null then 1 else 0 end) as visited 
        from hh_control ctrl
        left join n_hh_control_visited visited on ctrl.mapped_tv2_hhid = visited.mapped_tv2_hhid
        ),
    ```
    </font>
    
    - Exposed group
    <font size="1">
     ```
        hh_impressed_visited as (
        select
            impressed.mapped_tv2_hhid 
            ,(case when visited.mapped_tv2_hhid is not null then 1 else 0 end) as visited 
        from hh_impressed_in_campaign impressed
        left join n_hh_impressed_visited visited on impressed.mapped_tv2_hhid = visited.mapped_tv2_hhid
        ),
    ```
    </font> 
    Note: for some strange reason this `visited` column works in the overall volume check but when saving the data frame for manipulation in python this column was not bringing the correct nunmbers so it was recoded in python pandas, the reason as to why this is not wroking has not been investigated yet.


    


In [ ]:
import os
currentdir = os.getcwd()
correctdir = currentdir.rsplit('tv2ds/',1)[0]
os.chdir(correctdir)

from tv2ds.ds_lib import notebook_prodrun
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tvsquared.settings
from tvsquared.lib.request import Request
from tvsquared.lib.athena import AthenaDatabase
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)

In [ ]:
notebook_prodrun.set_env('US', prodrunenv='PROD')
crosswalk_suffix = os.environ.get('TV2PRODRUNENV').lower()
crosswalk_suffix

'produsa'

In [ ]:
def make_request(clientid, brandid = 1, datefrom=False, dateto=False):
    if datefrom and dateto:
        request = Request(clientarg=False, brandarg=False, datesarg=False)
        datefrom = datetime.datetime.strptime(datefrom, '%Y-%m-%d')
        dateto = datetime.datetime.strptime(dateto, '%Y-%m-%d')
        request = request.init(partnerid=None, clientid=clientid, brandid=brandid,datefrom=datefrom, dateto=dateto,loglevel=-1, extargs=None, request=None, usespark=None, readPreference=None,prodrun=False)
        return(request)
    else:
        request = Request(clientarg=False, brandarg=False, datesarg=False)
        request = request.init(partnerid=None, clientid=clientid, brandid=brandid,datefrom=None, dateto=None,loglevel=-1, extargs=None, request=None, usespark=None, readPreference=None,prodrun=False)
        return(request)


def query_athena(request, query, copy_to_local=False):
    athena_db = AthenaDatabase.get_client_database(request)
    query_results = AthenaDatabase.execute_query(athena_db, query, copy_to_local=copy_to_local)
    df = pd.DataFrame(query_results)
    return(df)

In [ ]:
client = {'clientid': 9306}
request = make_request(client['clientid'])

### Original query

In [ ]:
query = '''with filtered_hh as (
   select key_value as mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.crosswalk
   where yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='tv2_hhid' and vendor_name = '{vendor_name}' and excluded_stamp is null
),
campaign_universe as (
   select cw.mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.pre_crosswalk cw
   join filtered_hh fh on fh.mapped_tv2_hhid = cw.mapped_tv2_hhid
   where
       first_seen <= timestamp '{dateto}' + interval '1' day
       and last_seen >= timestamp '{datefrom}'
       and yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='ip_address' and vendor_name = '{vendor_name}'
),
client_eventlog as (
   select *
   from {crosswalk_suffix}_client_data.eventlog
   where client = '{clientid}'
),
hh_impressed_30days as (
   select
       distinct crosswalk_link_id as mapped_tv2_hhid
   from campaign_universe ex
   join client_eventlog ev on ev.crosswalk_link_id = ex.mapped_tv2_hhid
   where
       event_class='impression'
       and datadatetime between timestamp '{datefrom}' - interval '30' day    
       and timestamp '{dateto}' + interval '1' day + interval '6' day
       and in_scope
),
hh_impressed_in_campaign as (
   select
       distinct crosswalk_link_id as mapped_tv2_hhid
   from campaign_universe ex
   join client_eventlog ev on ev.crosswalk_link_id = ex.mapped_tv2_hhid
   where
       event_class='impression'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '1' day
       and in_scope
),
hh_control as (
   select
       distinct cu.mapped_tv2_hhid
   from campaign_universe cu
   left join hh_impressed_30days hh on hh.mapped_tv2_hhid  = cu.mapped_tv2_hhid
   where hh.mapped_tv2_hhid is null
),
n_hh_impressed_visited as (
   select count(distinct mapped_tv2_hhid) as exp_visited
   from client_eventlog
   join hh_impressed_in_campaign on crosswalk_link_id = mapped_tv2_hhid
   where
       event_class ='response' and event= 'all response'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '6' day + interval '1' day
       and in_scope
),
n_hh_control_visited as (
   select count(distinct mapped_tv2_hhid) as ctrl_visited
   from client_eventlog
   join hh_control on crosswalk_link_id = mapped_tv2_hhid
   where
       event_class ='response' and event= 'all response'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '6' day + interval '1' day
       and in_scope
),
n_hh_impressed_in_campaign as (
   select CAST(count(distinct mapped_tv2_hhid) AS double) as exp_hh
   from hh_impressed_in_campaign
),
n_hh_control as (
   select CAST(count(distinct mapped_tv2_hhid) AS double) as ctrl_hh
   from hh_control
),
final_results as (
   select *
   from n_hh_impressed_visited
   cross join n_hh_control_visited
   cross join n_hh_impressed_in_campaign
   cross join n_hh_control
)
select
   exp_visited,
   exp_hh,
   ctrl_visited,
   ctrl_hh,
   exp_visited/exp_hh as exp_vr,
   ctrl_visited/ctrl_hh as ctrl_vr,
   (exp_visited/exp_hh - ctrl_visited/ctrl_hh)/ (ctrl_visited/ctrl_hh) as uplift
from final_results

'''.format(
   vendor_name='inscape', 
   yy='2022', mm='03',dd='28',
   crosswalk_suffix='produsa',
   dateto='2022-03-31', 
   datefrom='2022-03-01', 
   clientid='9306') 

In [ ]:
df_filtered_hh= query_athena(request, query) # --> 6m to query

In [ ]:
df_filtered_hh

,exp_visited,exp_hh,ctrl_visited,ctrl_hh,exp_vr,ctrl_vr,uplift
0,19293,1137786.0,56959,4999056.0,0.016956615743206543,0.011393951177982403,0.488212076594939


### Query for control group

In [ ]:
query_control_march = '''
with filtered_hh as (
   select key_value as mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.crosswalk
   where yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='tv2_hhid' and vendor_name = '{vendor_name}' and excluded_stamp is null
),
campaign_universe as (
   select cw.mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.pre_crosswalk cw
   join filtered_hh fh on fh.mapped_tv2_hhid = cw.mapped_tv2_hhid
   where
       first_seen <= timestamp '{dateto}' + interval '1' day
       and last_seen >= timestamp '{datefrom}'
       and yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='ip_address' and vendor_name = '{vendor_name}'
),
client_eventlog as (
   select *
   from {crosswalk_suffix}_client_data.eventlog
   where client = '{clientid}'
),
hh_impressed_30days as (
   select
       distinct crosswalk_link_id as mapped_tv2_hhid
   from campaign_universe ex
   join client_eventlog ev on ev.crosswalk_link_id = ex.mapped_tv2_hhid
   where
       event_class='impression'
       and datadatetime between timestamp '{datefrom}' - interval '30' day    
       and timestamp '{dateto}' + interval '1' day + interval '6' day
       and in_scope
),
hh_control as (
   select
       distinct cu.mapped_tv2_hhid
   from campaign_universe cu
   left join hh_impressed_30days hh on hh.mapped_tv2_hhid  = cu.mapped_tv2_hhid
   where hh.mapped_tv2_hhid is null
),
n_hh_control_visited as (
   select distinct 
      mapped_tv2_hhid
   from client_eventlog
   join hh_control on crosswalk_link_id = mapped_tv2_hhid
   where
       event_class ='response' and event= 'all response'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '6' day + interval '1' day
       and in_scope
),
hh_control_visited as (
   select
      ctrl.mapped_tv2_hhid 
      ,(case when visited.mapped_tv2_hhid is not null then 1 else 0 end) as visited 
   from hh_control ctrl
   left join n_hh_control_visited visited on ctrl.mapped_tv2_hhid = visited.mapped_tv2_hhid
),
n_visited_control as (
   select SUM(visited) as ctrl_visited
   from hh_control_visited
),
n_hh_control as (
   select CAST(count(distinct mapped_tv2_hhid) AS double) as ctrl_hh
   from hh_control_visited
),
final_results as (
   select *
   from n_hh_control
   cross join n_visited_control
)
select
   *
from final_results

'''.format(vendor_name='inscape', yy='2022', mm='03',dd='28',crosswalk_suffix='produsa',dateto='2022-03-31', datefrom='2022-03-01', clientid='9306') 

In [ ]:
# the important table from above is: hh_control_visited
df_control_march= query_athena(request, query_control_march) #--> 5m to run

In [ ]:
df_control_march

,ctrl_hh,ctrl_visited
0,4999056.0,56959


#### Query for control group - adding date variables

In [ ]:
# Trying to bring date vars, but still not working ...
query_control_perday_march = '''
with filtered_hh as (
   select key_value as mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.crosswalk
   where yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='tv2_hhid' and vendor_name = '{vendor_name}' and excluded_stamp is null
),
campaign_universe as (
   select cw.mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.pre_crosswalk cw
   join filtered_hh fh on fh.mapped_tv2_hhid = cw.mapped_tv2_hhid
   where
       first_seen <= timestamp '{dateto}' + interval '1' day
       and last_seen >= timestamp '{datefrom}'
       and yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='ip_address' and vendor_name = '{vendor_name}'
),
client_eventlog as (
   select *
   from {crosswalk_suffix}_client_data.eventlog
   where client = '{clientid}'
),
hh_impressed_30days as (
   select
       distinct crosswalk_link_id as mapped_tv2_hhid
   from campaign_universe ex
   join client_eventlog ev on ev.crosswalk_link_id = ex.mapped_tv2_hhid
   where
       event_class='impression'
       and datadatetime between timestamp '{datefrom}' - interval '30' day    
       and timestamp '{dateto}' + interval '1' day + interval '6' day
       and in_scope
),
hh_control as (
   select
       distinct cu.mapped_tv2_hhid
   from campaign_universe cu
   left join hh_impressed_30days hh on hh.mapped_tv2_hhid  = cu.mapped_tv2_hhid
   where hh.mapped_tv2_hhid is null
),
n_hh_control_visited_per_day as (
   select  
      mapped_tv2_hhid
      ,date_trunc('day', datadatetime) as day
      ,event_class ='response' and event= 'all response'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '6' day + interval '1' day
       and in_scope as is_visit
   from client_eventlog
   join hh_control on crosswalk_link_id = mapped_tv2_hhid
),
hh_control_visited_per_day as (
   select 
   mapped_tv2_hhid
   ,day
   ,sum(is_visit) as n_visits
   ,sum(is_visit) > 0 as visited
   from  n_hh_control_visited_per_day
   group by mapped_tv2_hhid, day
),
n_visited_control as (
   select 
   SUM(n_visits) as ctrl_visits
   ,SUM(visited) as ctrl_visited
   from hh_control_visited_per_day
),
n_hh_control as (
   select CAST(count(distinct mapped_tv2_hhid) AS double) as ctrl_hh
   from hh_control_visited_per_day
),
final_results as (
   select *
   from n_hh_control
   cross join n_visited_control
)
select
   *
from final_results


'''.format(vendor_name='inscape', yy='2022', mm='03',dd='28',crosswalk_suffix='produsa',dateto='2022-03-31', datefrom='2022-03-01', clientid='9306') 

In [ ]:
df_control_perday_march= query_athena(request, query_control_perday_march) #--> 5m to run

Exception: ('Athena query failed', 'REASON: SYNTAX_ERROR: line 52:5: Unexpected parameters (boolean) for function sum. Expected: sum(double) , sum(real) , sum(bigint) , sum(interval day to second) , sum(interval year to month) , sum(decimal(p,s))   {\'QueryExecution\': {\'QueryExecutionId\': \'dbacccc7-441f-4ef8-8885-846d952df866\', \'Query\': "with filtered_hh as (\\n   select key_value as mapped_tv2_hhid\\n   from produsa_modeldata.crosswalk\\n   where yy=\'2022\' and mm=\'03\' and dd=\'28\' and key_name =\'tv2_hhid\' and vendor_name = \'inscape\' and excluded_stamp is null\\n),\\ncampaign_universe as (\\n   select cw.mapped_tv2_hhid\\n   from produsa_modeldata.pre_crosswalk cw\\n   join filtered_hh fh on fh.mapped_tv2_hhid = cw.mapped_tv2_hhid\\n   where\\n       first_seen <= timestamp \'2022-03-31\' + interval \'1\' day\\n       and last_seen >= timestamp \'2022-03-01\'\\n       and yy=\'2022\' and mm=\'03\' and dd=\'28\' and key_name =\'ip_address\' and vendor_name = \'inscape\'\

In [ ]:
df_control_perday_march

,ctrl_hh,ctrl_visited,ctrl_visited_day
0,4999056.0,155937,155937


### Query for exposed group

In [ ]:
query_exposed_march = '''with filtered_hh as (
   select key_value as mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.crosswalk
   where yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='tv2_hhid' and vendor_name = '{vendor_name}' and excluded_stamp is null
),
campaign_universe as (
   select cw.mapped_tv2_hhid
   from {crosswalk_suffix}_modeldata.pre_crosswalk cw
   join filtered_hh fh on fh.mapped_tv2_hhid = cw.mapped_tv2_hhid
   where
       first_seen <= timestamp '{dateto}' + interval '1' day
       and last_seen >= timestamp '{datefrom}'
       and yy='{yy}' and mm='{mm}' and dd='{dd}' and key_name ='ip_address' and vendor_name = '{vendor_name}'
),
client_eventlog as (
   select *
   from {crosswalk_suffix}_client_data.eventlog
   where client = '{clientid}'
),
hh_impressed_in_campaign as (
   select
       distinct crosswalk_link_id as mapped_tv2_hhid
   from campaign_universe ex
   join client_eventlog ev on ev.crosswalk_link_id = ex.mapped_tv2_hhid
   where
       event_class='impression'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '1' day
       and in_scope
),
n_hh_impressed_visited as (
   select distinct 
      mapped_tv2_hhid
   from client_eventlog
   join hh_impressed_in_campaign on crosswalk_link_id = mapped_tv2_hhid
   where
       event_class ='response' and event= 'all response'
       and datadatetime between timestamp '{datefrom}' and timestamp '{dateto}' + interval '6' day + interval '1' day
       and in_scope
),
hh_impressed_visited as (
   select
      impressed.mapped_tv2_hhid 
      ,(case when visited.mapped_tv2_hhid is not null then 1 else 0 end) as visited 
   from hh_impressed_in_campaign impressed
   left join n_hh_impressed_visited visited on impressed.mapped_tv2_hhid = visited.mapped_tv2_hhid
),
n_visited_impressed as (
   select SUM(visited) as exp_visited
   from hh_impressed_visited
),
n_hh_impressed as (
   select CAST(count(distinct mapped_tv2_hhid) AS double) as exp_hh
   from hh_impressed_visited
),
final_results as (
   select *
   from n_hh_impressed
   cross join n_visited_impressed
)
select
   *
from final_results

'''.format(vendor_name='inscape', yy='2022', mm='03',dd='28',crosswalk_suffix='produsa',dateto='2022-03-31', datefrom='2022-03-01', clientid='9306') 

In [ ]:
df_exposed_march= query_athena(request, query_exposed_march) #--> 6m to run 

In [ ]:
df_exposed_march

,exp_hh,exp_visited
0,1137786.0,19293
